In [1]:
import os
import sys
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sys.path.append("../gotmtool")
from gotmtool import *

## Create a model
Create a model with environment file `../../.gotm_env.yaml`, which is created by `gotm_env_init.py`. 

In [2]:
casename = 'ERA5_plus_STD-2h'
# datadir = '/Volumes/WD02/SaildroneFlux/data'
# m = Model(name=casename, environ='../gotmtool/.gotm_env_wd02.yaml')
datadir = '/Users/qingli/scratch/saildrone_southern_ocean/'+casename
m = Model(name=casename, environ='../gotmtool/.gotm_env.yaml')

Take a look at what are defined in the environment file.

In [3]:
for key in m.environ:
    print('{:>15s}: {}'.format(key, m.environ[key]) )

   gotmdir_code: /Users/qingli/scratch/saildrone_southern_ocean/gotm/code
   gotmdir_data: /Users/qingli/scratch/saildrone_southern_ocean/gotm/data
  gotmdir_build: /Users/qingli/scratch/saildrone_southern_ocean/gotm/build
    gotmdir_exe: /Users/qingli/scratch/saildrone_southern_ocean/gotm/exe
    gotmdir_run: /Users/qingli/scratch/saildrone_southern_ocean/gotm/run
 gotmdir_figure: /Users/qingli/scratch/saildrone_southern_ocean/gotm/figure
   gotmdir_tool: /Users/qingli/scratch/saildrone_southern_ocean/gotmtool


## Build the model

In [4]:
%%time
m.build()

CPU times: user 1.19 ms, sys: 7.88 ms, total: 9.07 ms
Wall time: 64.9 ms


## Configuration
Initialize the GOTM configuration

In [5]:
cfgfile = m.environ['gotmdir_run']+'/'+casename+'/gotm.yaml'
if os.path.exists(cfgfile):
    print('loading configure file...')
    cfg = yaml_load(filename=cfgfile)
else:
    cfg = m.init_config(filename=cfgfile)

loading configure file...


Update the configuration

In [6]:
# setup
title = 'Constant forcing'
nlev = 400
cfg['title'] = title
cfg['location']['name'] = 'Idealized'
cfg['location']['depth'] = 200
cfg['time']['dt']    = 60.0
cfg['grid']['nlev']  = nlev

# output
cfg['output'] = {}
cfg['output']['gotm_out'] = {}
cfg['output']['gotm_out']['use'] = True
cfg['output']['gotm_out']['title'] = title
cfg['output']['gotm_out']['k1_stop'] = nlev+1
cfg['output']['gotm_out']['k_stop'] = nlev
cfg['output']['gotm_out']['time_unit'] = 'dt'
cfg['output']['gotm_out']['time_step'] = 60
cfg['output']['gotm_out']['variables'] = [{}]
cfg['output']['gotm_out']['variables'][0]['source'] = '*'

# forcing
cfg['temperature']['method'] = 'file'
cfg['salinity']['method'] = 'file'
cfg['surface']['fluxes']['heat']['method'] = 'constant'
cfg['surface']['fluxes']['tx']['method'] = 'constant'
cfg['surface']['fluxes']['ty']['method'] = 'constant'
cfg['surface']['fluxes']['ty']['constant_value'] = 0.0
cfg['surface']['swr']['method'] = 'file'
cfg['surface']['swr']['file'] = datadir+'/swr_era5.dat'
cfg['surface']['precip']['method'] = 'constant'
cfg['waves']['stokes_drift']['us']['method'] = 'off'
cfg['waves']['stokes_drift']['vs']['method'] = 'off'

# EOS -- use linear
cfg['eq_state']['form'] = 'full-pot'

# water type (Jerlov IB)
cfg['light_extinction']['method'] = 'jerlov-ib'

In [7]:
cfg['turbulence']['turb_method'] = 'second_order'
cfg['turbulence']['tke_method'] = 'tke'
cfg['turbulence']['len_scale_method'] = 'gls'
cfg['turbulence']['scnd']['method'] =  'weak_eq_kb_eq'
cfg['turbulence']['scnd']['scnd_coeff'] =  'canuto-a'
cfg['turbulence']['turb_param']['length_lim'] = 'false'
cfg['turbulence']['turb_param']['compute_c3'] = 'true'
cfg['turbulence']['turb_param']['Ri_st'] = 0.25
cfg['turbulence']['generic']['gen_m'] = 1.5 
cfg['turbulence']['generic']['gen_n'] = -1.0
cfg['turbulence']['generic']['gen_p'] = 3.0 
cfg['turbulence']['generic']['cpsi1'] = 1.44
cfg['turbulence']['generic']['cpsi2'] = 1.92
cfg['turbulence']['generic']['cpsi3minus'] = -0.63
cfg['turbulence']['generic']['cpsi3plus'] = 1.0 
cfg['turbulence']['generic']['sig_kpsi'] = 1.0 
cfg['turbulence']['generic']['sig_psi'] = 1.3

In [8]:
# collect the configurations and the labels of the two runs
cfgs = []
labels = []

In [9]:
# load list of cases
with open(os.path.join(datadir, 'caselist.txt'), 'r') as f:
    lines = f.readlines()
caselist = [line.strip() for line in lines]
caselist

['C0024',
 'C0028',
 'C0032',
 'C0036',
 'C0040',
 'C0044',
 'C0048',
 'C0052',
 'C0056',
 'C0060',
 'C0064',
 'C0068',
 'C0072',
 'C0076',
 'C0080',
 'C0084',
 'C0088',
 'C0092',
 'C0096',
 'C0100',
 'C0104',
 'C0108',
 'C0112',
 'C0116',
 'C0120',
 'C0124',
 'C0128',
 'C0132',
 'C0136',
 'C0140',
 'C0144',
 'C0148',
 'C0152',
 'C0156',
 'C0160',
 'C0164',
 'C0168',
 'C0172',
 'C0176',
 'C0180',
 'C0184',
 'C0188',
 'C0192',
 'C0196',
 'C0200',
 'C0204',
 'C0208',
 'C0212',
 'C0216',
 'C0220',
 'C0224',
 'C0228',
 'C0232',
 'C0236',
 'C0240',
 'C0244',
 'C0248',
 'C0252',
 'C0256',
 'C0260',
 'C0264',
 'C0268',
 'C0272',
 'C0276',
 'C0280',
 'C0284',
 'C0288',
 'C0292',
 'C0296',
 'C0300',
 'C0304',
 'C0308',
 'C0312',
 'C0316',
 'C0320',
 'C0324',
 'C0328',
 'C0332',
 'C0336',
 'C0340',
 'C0344',
 'C0348',
 'C0352',
 'C0356',
 'C0360',
 'C0364',
 'C0368',
 'C0372',
 'C0376',
 'C0380',
 'C0384',
 'C0388',
 'C0392',
 'C0396',
 'C0400',
 'C0404',
 'C0408',
 'C0412',
 'C0416',
 'C0420',


In [10]:
nstd_hf  = [-3., -2., -1., 0., 1., 2., 3.]
nstd_tau = [-3., -2., -1., 0., 1., 2., 3.]
nstd_hf_label = ['{prefix}{val:d}'.format(prefix='p' if v > 0 else 'm' if v < 0 else '',
                                          val=abs(int(v))) for v in nstd_hf]
nstd_tau_label = ['{prefix}{val:d}'.format(prefix='p' if v > 0 else 'm' if v < 0 else '',
                                           val=abs(int(v))) for v in nstd_tau]
stau = 0.18
sqlat = 0.09
sqsens = 0.11
record = ''
for cname in caselist:
    configfile = os.path.join(datadir, cname, 'config.dat')
    tproffile = os.path.join(datadir, cname, 't_prof.dat')
    sproffile = os.path.join(datadir, cname, 's_prof.dat')
    with open(configfile, 'r') as f:
        lines = f.readlines()
    startdate = lines[0].strip()
    lon, lat, hf, tau, qlat, qsens = lines[1].strip().split()
    lon = float(lon)
    lat = float(lat)
    hf = float(hf)
    tau = float(tau)
    qlat = float(qlat)
    qsens = float(qsens)
    dttime = pd.to_datetime(startdate)
    dttime_stop = dttime + pd.DateOffset(days=4)
    stopdate = dttime_stop.strftime('%Y-%m-%d %H:%M:%S')
    print(cname)
    record += '{}: {} -> {}\n'.format(cname, startdate, stopdate)
    record += '  lon: {} lat: {}\n'.format(lon, lat)
    record += '  Qᵣ: {:6.2f}  Qˢᵣ: {:6.2f}  Qˡᵣ: {:6.2f}  τᵣ: {:5.3f}\n'.format(hf, qsens, qlat, tau)
    record += '-'*64+'\n'
    for i, std1 in enumerate(nstd_hf):
        hf_a  = hf + std1*(qsens*sqsens+qlat*sqlat)
        plist = []
        for j, std2 in enumerate(nstd_tau):
            tau_a = tau + std2*tau*stau
            plist.append('{:6.2f}, {:5.3f}'.format(hf_a, tau_a))
            rlabel = '-'.join([cname, 'hf', nstd_hf_label[i], 'tau', nstd_tau_label[j]])
#             print(clabel)
            # set config
            cfg['location']['latitude'] = lat
            cfg['location']['longitude'] = lon
            cfg['time']['start'] = startdate
            cfg['time']['stop']  = stopdate
            cfg['turbulence']['turb_method'] = 'second_order' 
            cfg['temperature']['file'] = tproffile
            cfg['salinity']['file'] = sproffile
            cfg['surface']['fluxes']['heat']['constant_value'] = hf_a
            cfg['surface']['fluxes']['tx']['constant_value'] = tau_a
            cfgs.append(copy.deepcopy(cfg))
            labels.append(rlabel+'_GLS-C01A')
            cfg['turbulence']['turb_method'] = 'cvmix'
            cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 'none'
            cfgs.append(copy.deepcopy(cfg))
            labels.append(rlabel+'_KPP-CVMix')
        record += ' | '.join(plist)+'\n'         
    record += '\n'
with open(os.path.join(datadir, 'forcing_summary.log'), 'w') as f:
    f.write(record)

C0024
C0028
C0032
C0036
C0040
C0044
C0048
C0052
C0056
C0060
C0064
C0068
C0072
C0076
C0080
C0084
C0088
C0092
C0096
C0100
C0104
C0108
C0112
C0116
C0120
C0124
C0128
C0132
C0136
C0140
C0144
C0148
C0152
C0156
C0160
C0164
C0168
C0172
C0176
C0180
C0184
C0188
C0192
C0196
C0200
C0204
C0208
C0212
C0216
C0220
C0224
C0228
C0232
C0236
C0240
C0244
C0248
C0252
C0256
C0260
C0264
C0268
C0272
C0276
C0280
C0284
C0288
C0292
C0296
C0300
C0304
C0308
C0312
C0316
C0320
C0324
C0328
C0332
C0336
C0340
C0344
C0348
C0352
C0356
C0360
C0364
C0368
C0372
C0376
C0380
C0384
C0388
C0392
C0396
C0400
C0404
C0408
C0412
C0416
C0420
C0424
C0428
C0432
C0436
C0440
C0444
C0448
C0452
C0456
C0460
C0464
C0468
C0472
C0476
C0480
C0484
C0488
C0492
C0496
C0500
C0504
C0508
C0512
C0516
C0520
C0524
C0528
C0532
C0536
C0540
C0544
C0548
C0552
C0556
C0560
C0564
C0568
C0572
C0576
C0580
C0584
C0588
C0592
C0596
C0600
C0604
C0608
C0612
C0616
C0620
C0624
C0628
C0632
C0636
C0640
C0644
C0648
C0652
C0656
C0660
C0664
C0668
C0672
C0676
C0680
C0684
C068

## Run the model

In [11]:
%%time
sims = m.run_batch(configs=cfgs, labels=labels, nproc=4)

CPU times: user 28.5 s, sys: 7.59 s, total: 36.1 s
Wall time: 1h 23min 41s
